In [2]:
import pandas as pd
import spacy
import seaborn as sn
import matplotlib.pyplot as plt


nlp = spacy.load("en_core_web_lg")


In [3]:
df = pd.read_csv("Emotion_classify_Data.csv")
df.head()

,Comment,Emotion
0,i seriously hate one subject to death but now ...,fear
1,im so full of life i feel appalled,anger
2,i sit here to write i start to dig out my feel...,fear
3,ive been really angry with r and i feel like a...,joy
4,i feel suspicious if there is no one outside l...,fear


In [4]:
df.shape

(5937, 2)

In [5]:
df.isnull().sum()

Comment    0
Emotion    0
dtype: int64

In [6]:
df.Emotion.value_counts()

Emotion
anger    2000
joy      2000
fear     1937
Name: count, dtype: int64

In [7]:
def preprocess(text):
    filtered_tokens = []

    word = nlp(text)

    for token in word:
        if token.is_stop or token.is_punct:
            continue
        filtered_tokens.append(token.lemma_)
    
    return " ".join(filtered_tokens)

In [8]:
df["new_label"] = df.Emotion.map({
    "anger" : 0,
    'joy' : 1,
    "fear" : 2
})

In [9]:
df["Preprocessed_comment"] = df.Comment.apply(preprocess)

In [10]:
df.head()

,Comment,Emotion,new_label,Preprocessed_comment
0,i seriously hate one subject to death but now ...,fear,2,seriously hate subject death feel reluctant drop
1,im so full of life i feel appalled,anger,0,m life feel appalled
2,i sit here to write i start to dig out my feel...,fear,2,sit write start dig feeling think afraid accep...
3,ive been really angry with r and i feel like a...,joy,1,ve angry r feel like idiot trust place
4,i feel suspicious if there is no one outside l...,fear,2,feel suspicious outside like rapture happen


In [11]:
from sklearn.model_selection import train_test_split
x_tr, x_t, y_tr, y_t = train_test_split(df.Preprocessed_comment,
                                        df.new_label,
                                        test_size=0.2)

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
model2 = Pipeline([
    ("tdif", TfidfVectorizer()),
    ("log", LogisticRegression())
])

model2.fit(x_tr, y_tr)


Pipeline(steps=[('tdif', TfidfVectorizer()), ('log', LogisticRegression())])

In [13]:
y_pred2 = model2.predict(x_t)
print(classification_report(y_t, y_pred2))

              precision    recall  f1-score   support

           0       0.93      0.95      0.94       399
           1       0.93      0.96      0.95       423
           2       0.97      0.91      0.94       366

    accuracy                           0.94      1188
   macro avg       0.94      0.94      0.94      1188
weighted avg       0.94      0.94      0.94      1188



In [14]:
from sklearn.naive_bayes import MultinomialNB


clf2 = Pipeline([
     ('vectorizer_tfidf',TfidfVectorizer()),    
     ('Multi NB', MultinomialNB())         
])

clf2.fit(x_tr, y_tr)


y_pred3 = clf2.predict(x_t)

print(classification_report(y_t
                            , y_pred3))

              precision    recall  f1-score   support

           0       0.89      0.93      0.91       399
           1       0.94      0.90      0.92       423
           2       0.89      0.90      0.90       366

    accuracy                           0.91      1188
   macro avg       0.91      0.91      0.91      1188
weighted avg       0.91      0.91      0.91      1188



In [15]:
re = model2.predict(["I am very sad and unhappy"])
if re == 0:
    print("Emotion: Anger")
elif re == 1:
    print("Emotion: Joy")
else:
    print("Emotion: Fear")

Emotion: Anger


In [16]:
re = clf2.predict(["I am very sad and unhappy"])
if re == 0:
    print("Emotion: Anger")
elif re == 1:
    print("Emotion: Joy")
else:
    print("Emotion: Fear")

Emotion: Anger


In [17]:
re = model2.predict(["I am very glad i got the job... yay"])
if re == 0:
    print("Emotion: Anger")
elif re == 1:
    print("Emotion: Joy")
else:
    print("Emotion: Fear")

Emotion: Joy


In [18]:
re = clf2.predict(["I am very glad i got the job... yay"])
if re == 0:
    print("Emotion: Anger")
elif re == 1:
    print("Emotion: Joy")
else:
    print("Emotion: Fear")

Emotion: Joy


In [20]:
re = model2.predict(["I dont want to go home, i am afraid my mum will beat me"])
if re == 0:
    print("Emotion: Anger")
elif re == 1:
    print("Emotion: Joy")
else:
    print("Emotion: Fear")

Emotion: Fear


In [19]:
re = clf2.predict(["I dont want to go home, i am afraid my mum will beat me"])
if re == 0:
    print("Emotion: Anger")
elif re == 1:
    print("Emotion: Joy")
else:
    print("Emotion: Fear")

Emotion: Fear


In [ ]:
import joblib 
joblib.dump(model2, "emotion_detection")